In [1]:
import pandas as pd
import numpy as np
import torch

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import tensorflow as tf

from helper_methods import *

from main import create_model

# define path to glove and snli files
path = "../Project/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
snli_train = pd.read_csv(path+'snli_1.0/snli_1.0_train.txt', sep="\\t")
snli_test = pd.read_csv(path+'snli_1.0/snli_1.0_test.txt', sep="\\t")

C:\Users\Danie\Anaconda3\envs\gameAI\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\Users\Danie\Anaconda3\envs\gameAI\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
print(snli_train['sentence1'][0],snli_train['sentence2'][0], snli_train['gold_label'][0])
print(snli_train['sentence1'][1],snli_train['sentence2'][1], snli_train['gold_label'][1])
print(snli_train['sentence1'][2],snli_train['sentence2'][2], snli_train['gold_label'][2])

A person on a horse jumps over a broken down airplane. A person is training his horse for a competition. neutral
A person on a horse jumps over a broken down airplane. A person is at a diner, ordering an omelette. contradiction
A person on a horse jumps over a broken down airplane. A person is outdoors, on a horse. entailment


In [4]:
# Filter and rename the important columns
train_df = snli_train.filter(['sentence1','sentence2','gold_label'], axis=1)
test_df = snli_test.filter(['sentence1','sentence2','gold_label'], axis=1)
train_df = train_df.rename(columns={"sentence1": "premise", "sentence2": "hypothesis"})
test_df = test_df.rename(columns={"sentence1": "premise", "sentence2": "hypothesis"})

# exclude sentences which could not be categorised
train_df = train_df[train_df['gold_label'] != "-"]
test_df = test_df[test_df['gold_label'] != "-"]

train_df['gold_label'].value_counts()

entailment       183416
contradiction    183187
neutral          182764
Name: gold_label, dtype: int64

# Glove

In [5]:
embeddings_dict = get_glove_embedding(path)

# Word embeddings

In [6]:
pad_seq_prem, embedding_matrix_prem, vocab_size_prem = get_embedding_matrix(train_df['premise'], embeddings_dict)

A person on a horse jumps over a broken down airplane.
[1, 56, 5, 1, 197, 191, 68, 1, 1762, 36, 877]
the max sentence length is: 78
18490
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  59 132   4 666  16  94]


In [7]:
pad_seq_hyp, embedding_matrix_hyp, vocab_size_hyp = get_embedding_matrix(train_df['hypothesis'], embeddings_dict)

A person is training his horse for a competition.
[1, 27, 3, 821, 16, 162, 20, 1, 275]
the max sentence length is: 56
30904
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0  22   4  40 705]


In [8]:
enc_gold_label = encode_labels(train_df['gold_label'])

[array(['contradiction', 'entailment', 'neutral'], dtype=object)]
[0. 1. 0.]


# Sentence Embeddings

## Baseline Embeddings(Use sum of word embeddings to get sentence embedding)

In [9]:
sentence_embs = baseline_sum_sentence_embeddings(pad_seq_prem, embedding_matrix_prem)

In [10]:
sentence_embs_hyp = baseline_sum_sentence_embeddings(pad_seq_hyp, embedding_matrix_hyp)

# Model

In [11]:
model = create_model()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 100)          30100       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 100)          30100       input_2[0][0]                    
_______________________________________________________________________________________

# Train

In [12]:
model.compile(optimizer='Adadelta',loss='categorical_crossentropy',metrics = ['accuracy'])

In [13]:
history = model.fit([np.array(sentence_embs), np.array(sentence_embs_hyp)],enc_gold_label,epochs = 100,batch_size=256,validation_split=0.2, verbose=1)

Epoch 1/100
1717/1717 [==============================] - 8s 5ms/step - loss: 1.1271 - accuracy: 0.3444 - val_loss: 1.1064 - val_accuracy: 0.3639
Epoch 2/100
1717/1717 [==============================] - 8s 4ms/step - loss: 1.0931 - accuracy: 0.3857 - val_loss: 1.0821 - val_accuracy: 0.4039
Epoch 3/100
1717/1717 [==============================] - 8s 5ms/step - loss: 1.0732 - accuracy: 0.4199 - val_loss: 1.0656 - val_accuracy: 0.4337
Epoch 4/100
1717/1717 [==============================] - 9s 5ms/step - loss: 1.0583 - accuracy: 0.4440 - val_loss: 1.0523 - val_accuracy: 0.4535
Epoch 5/100
1717/1717 [==============================] - 9s 5ms/step - loss: 1.0461 - accuracy: 0.4603 - val_loss: 1.0411 - val_accuracy: 0.4668
Epoch 6/100
1717/1717 [==============================] - 8s 5ms/step - loss: 1.0356 - accuracy: 0.4726 - val_loss: 1.0315 - val_accuracy: 0.4767
Epoch 7/100
1717/1717 [==============================] - 8s 5ms/step - loss: 1.0268 - accuracy: 0.4814 - val_loss: 1.0233 - val_ac

1717/1717 [==============================] - 9s 5ms/step - loss: 0.9238 - accuracy: 0.5638 - val_loss: 0.9236 - val_accuracy: 0.5644
Epoch 58/100
1717/1717 [==============================] - 9s 5ms/step - loss: 0.9229 - accuracy: 0.5644 - val_loss: 0.9227 - val_accuracy: 0.5655
Epoch 59/100
1717/1717 [==============================] - 10s 6ms/step - loss: 0.9221 - accuracy: 0.5650 - val_loss: 0.9219 - val_accuracy: 0.5656
Epoch 60/100
1717/1717 [==============================] - 9s 5ms/step - loss: 0.9213 - accuracy: 0.5657 - val_loss: 0.9211 - val_accuracy: 0.5665
Epoch 61/100
1717/1717 [==============================] - 8s 5ms/step - loss: 0.9205 - accuracy: 0.5663 - val_loss: 0.9203 - val_accuracy: 0.5673
Epoch 62/100
1717/1717 [==============================] - 9s 5ms/step - loss: 0.9196 - accuracy: 0.5668 - val_loss: 0.9195 - val_accuracy: 0.5677
Epoch 63/100
1717/1717 [==============================] - 9s 5ms/step - loss: 0.9188 - accuracy: 0.5674 - val_loss: 0.9188 - val_accurac

# Test

In [14]:
pad_seq_prem_test, embedding_matrix_prem_test, vocab_size_prem_test = get_embedding_matrix(test_df['premise'], embeddings_dict)
pad_seq_hyp_test, embedding_matrix_hyp_test, vocab_size_hyp_test = get_embedding_matrix(test_df['hypothesis'], embeddings_dict)

sentence_embs_prem_test = baseline_sum_sentence_embeddings(pad_seq_prem_test, embedding_matrix_prem_test)
sentence_embs_hyp_test = baseline_sum_sentence_embeddings(pad_seq_hyp_test, embedding_matrix_hyp_test)

enc_gold_label_test = encode_labels(test_df['gold_label'])

This church choir sings to the masses as they sing joyous songs from the book at a church.
[281, 668, 779, 600, 12, 3, 2084, 42, 184, 1455, 2085, 2086, 64, 3, 272, 17, 1, 668]
the max sentence length is: 54
4093
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    1   10    9    1   43 1400   22   16    4    1  218
  282 2087]
The church has cracks in the ceiling.
[2, 655, 36, 2754, 6, 2, 2755]
the max sentence length is: 29
5166
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0

In [15]:
# Evaluate the model on the test data using evaluate
print("Evaluate on test data")
results = model.evaluate([sentence_embs_prem_test, sentence_embs_hyp_test], enc_gold_label_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
77/77 [==============================] - 0s 2ms/step - loss: 0.8877 - accuracy: 0.5857
test loss, test acc: [0.8876674771308899, 0.5857084393501282]


In [16]:
print("Generate predictions for 3 samples")
predictions = model.predict([sentence_embs_prem_test, sentence_embs_hyp_test])
print("predictions shape:", predictions.shape)

Generate predictions for 3 samples
predictions shape: (9824, 3)


In [17]:
print(predictions[2])
# order: contradiction, entailment, neutral

[0.669341   0.09758869 0.23307024]


In [18]:
print(test_df['premise'][2], test_df['hypothesis'][2], test_df['gold_label'][2])

This church choir sings to the masses as they sing joyous songs from the book at a church. A choir singing at a baseball game. contradiction
